In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/")

Mounted at /content/drive


In [ ]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import os
#数据加载
path = os.path.abspath(os.path.dirname(os.getcwd()) + os.path.sep + ".")
input_path = path + '/Colab Notebooks/'
Train_data = pd.read_csv(input_path+'used_car_train_20200313.csv', sep=' ')
Test_data = pd.read_csv(input_path+'used_car_testB_20200421.csv', sep=' ')
"""
—————————————————————————————————————————————以下为神经网络的数据处理—————————————————————————————————————————————
"""

# 合并方便后面的操作
df = pd.concat([Train_data, Test_data], ignore_index=True)


#选择需要使用的特征标签，由于nn会生成大量的特征，我们只需要保留原始特征和刻画几个明显特征即可
feature = ['model','brand','bodyType','fuelType','kilometer','notRepairedDamage','power','regDate_month','creatDate_year','creatDate_month'
    ,'v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_5', 'v_6',
       'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12', 'v_13', 'v_14','car_age_day','car_age_year','regDate_year','name_count']


#处理异常数据
df.drop(df[df['seller'] == 1].index, inplace=True)
#记录一下df的price
df_copy = df

df['power'][df['power']>600]=600

#notRepairedDamage的值是0和1，然后为-的值设置为0.5，在将它进行标签转换，0->1;0.5->2;1->3;这样符合神经网络的特征提取，不确定值位于两个确定值的中间～
df.replace(to_replace = '-', value = 0.5, inplace = True)
le = LabelEncoder()
df['notRepairedDamage'] = le.fit_transform(df['notRepairedDamage'].astype(str))

#日期处理
from datetime import datetime
def date_process(x):
    year = int(str(x)[:4])
    month = int(str(x)[4:6])
    day = int(str(x)[6:8])

    if month < 1:
        month = 1

    date = datetime(year, month, day)
    return date

df['regDates'] = df['regDate'].apply(date_process)
df['creatDates'] = df['creatDate'].apply(date_process)
df['regDate_year'] = df['regDates'].dt.year
df['regDate_month'] = df['regDates'].dt.month
df['regDate_day'] = df['regDates'].dt.day
df['creatDate_year'] = df['creatDates'].dt.year
df['creatDate_month'] = df['creatDates'].dt.month
df['creatDate_day'] = df['creatDates'].dt.day
df['car_age_day'] = (df['creatDates'] - df['regDates']).dt.days
df['car_age_year'] = round(df['car_age_day'] / 365, 1)

#对name进行挖掘
df['name_count'] = df.groupby(['name'])['SaleID'].transform('count')

#填充众数
df.fillna(df.median(),inplace= True)



#特征归一化
scaler = MinMaxScaler()
scaler.fit(df[feature].values)
df= scaler.transform(df[feature].values)


## 切割数据,导出数据,作为神经网络的训练数据
output_path = path + '/Colab Notebooks/'
nn_data = pd.DataFrame(df,columns=feature)
nn_data['price']=np.array(df_copy['price'])
nn_data['SaleID']=np.array(df_copy['SaleID'])
print(nn_data.shape)
train_num = df.shape[0]-50000
nn_data[0:int(train_num)].to_csv(output_path+'train_nn.csv', index=0, sep=' ')
nn_data[train_num:train_num+50000].to_csv(output_path+'test_nn.csv', index=0, sep=' ')

(199999, 31)


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import tensorflow.keras.backend as K
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import os
import keras
"""
神经网络
"""
## 读取神经网络模型数据

path = os.path.abspath(os.path.dirname(os.getcwd()) + os.path.sep + ".")
tree_data_path = path + '/Colab Notebooks/'
Train_NN_data = pd.read_csv(tree_data_path+'train_nn.csv', sep=' ')
Test_NN_data = pd.read_csv(tree_data_path+'test_nn.csv', sep=' ')

numerical_cols = Train_NN_data.columns
feature_cols = [col for col in numerical_cols if col not in ['price','SaleID']]
## 提前特征列，标签列构造训练样本和测试样本
X_data = Train_NN_data[feature_cols]
X_test = Test_NN_data[feature_cols]


x = np.array(X_data)
y = np.array(Train_NN_data['price'])
x_test = np.array(X_test)

# callbacks 是在训练过程中进行监听的，因此是在fit函数中对callbacks进行添加
# 定义一个callbacks的数组，然后在fit函数中将callbacks的数组当作参数传进去
logdir = './callbacks'
if not os.path.exists(logdir):
    os.mkdir(logdir)
logdir = os.path.join("callbacks") # 好像只是在window上要添加这行，否则出错

output_model_file = os.path.join(logdir,"fashion_mnist_model.h5")  # 输出的model文件

callbacks = [
    keras.callbacks.TensorBoard(logdir),
    keras.callbacks.ModelCheckpoint(output_model_file, save_best_only=True),
    keras.callbacks.EarlyStopping(patience=5, min_delta=1e-3),
]


kfolder = KFold(n_splits=5, shuffle=True, random_state=2018)
oof_nn = np.zeros(len(x))
predictions_nn = np.zeros(len(x_test))
predictions_train_nn = np.zeros(len(x))
kfold = kfolder.split(x, y)
fold_ = 0
for train_index, vali_index in kfold:
    k_x_train = x[train_index]
    k_y_train = y[train_index]
    k_x_vali = x[vali_index]
    k_y_vali = y[vali_index]

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.02)))
    model.add(tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.02)))
    model.add(tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.02)))
    model.add(tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.02)))
    model.add(tf.keras.layers.Dense(1, kernel_regularizer=tf.keras.regularizers.l2(0.02)))

    model.compile(loss='mean_absolute_error',
                    optimizer=tf.keras.optimizers.Adam(),
                  metrics=['mae'])

    model.fit(k_x_train,k_y_train,batch_size =512,epochs=2000,validation_data=(k_x_vali, k_y_vali), callbacks=callbacks)#callbacks=callbacks,
    oof_nn[vali_index] = model.predict(k_x_vali).reshape((model.predict(k_x_vali).shape[0],))
    predictions_nn += model.predict(x_test).reshape((model.predict(x_test).shape[0],)) / kfolder.n_splits
    predictions_train_nn += model.predict(x).reshape((model.predict(x).shape[0],)) / kfolder.n_splits

print("NN score: {:<8.8f}".format(mean_absolute_error(oof_nn, y)))

output_path = path + '/Colab Notebooks/'
# 测试集输出
predictions = predictions_nn
predictions[predictions < 0] = 0
sub = pd.DataFrame()
sub['SaleID'] = Test_NN_data.SaleID
sub['price'] = predictions
sub.to_csv(output_path+'nn_test.csv', index=False)

# 验证集输出
oof_nn[oof_nn < 0] = 0
sub = pd.DataFrame()
sub['SaleID'] = Train_NN_data.SaleID
sub['price'] = oof_nn
sub.to_csv(output_path+'nn_train.csv', index=False)

Epoch 1/2000
  1/235 [..............................] - ETA: 0s - loss: 5521.8818 - mae: 5508.7842WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
235/235 [==============================] - 5s 19ms/step - loss: 3228.9573 - mae: 3210.4705 - val_loss: 1318.7882 - val_mae: 1285.6323
Epoch 2/2000
235/235 [==============================] - 4s 17ms/step - loss: 1169.1245 - mae: 1131.1156 - val_loss: 1081.2191 - val_mae: 1040.4966
Epoch 3/2000
235/235 [==============================] - 4s 18ms/step - loss: 1017.2012 - mae: 974.7172 - val_loss: 967.2228 - val_mae: 923.3298
Epoch 4/2000
235/235 [==============================] - 4s 18ms/step - loss: 926.2065 - mae: 881.4611 - val_loss: 889.7217 - val_mae: 844.1917
Epoch 5/2000
235/235 [==============================] - 